* Setup Postgres Database
* Install required dependencies
* Reconfigure Application to Postgres
* Launch Flask Shell with Environment Variables
* Setup Table and Data using Flask Shell
* Run and Validate the Application
* Validate using pgAdmin
* Exercise and Solution

* Setup Postgres Database

Login as superuser postgres and create database by name sales_db and user sales_user.
```sql
CREATE DATABASE sales_db;
CREATE USER sales_user WITH ENCRYPTED PASSWORD 'itversity';
ALTER DATABASE sales_db OWNER TO sales_user;
COMMIT;
```

Make sure to disconnect postgres connection and validate by creating and connecting using new connection details.
```sql
SELECT current_date;
```

* Install required dependencies

Update `requirements.txt` and install using pip command. Make sure to change the versions depending up on your environment.
```text
Flask==2.2.3
Flask-SQLAlchemy==3.0.3
Flask-Bootstrap==3.3.7.1
psycopg2-binary==2.9.6
```

Run `pip install -r requirements.txt` to install all the required dependencies

* Reconfigure Application to Postgres

Update `app.py`.

```python
import os
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from flask_bootstrap import Bootstrap

db = SQLAlchemy()
app = Flask(__name__)
Bootstrap(app)

host = os.environ.get('SALES_DB_HOST')
port = os.environ.get('SALES_DB_PORT')
db_name = os.environ.get('SALES_DB_NAME')
user = os.environ.get('SALES_DB_USER')
password = os.environ.get('SALES_DB_PASS')
app.config["SQLALCHEMY_DATABASE_URI"] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)

from models.user import User


@app.route('/')
def hello_world():
    rec = db.get_or_404(User, 1)
    return render_template('index.html', user=rec)


@app.route('/users')
def users():
    user_recs = db.session.query(User).all()
    users = list(map(lambda rec: rec.__dict__, user_recs))
    return render_template('users.html', users=users)
```

Update `models/user.py`

```python
from app import db

class User(db.Model):
    __tablename__ = 'users' # Model and table name are different
    id = db.Column(db.Integer, primary_key=True)
    first_name = db.Column(db.String)
    last_name = db.Column(db.String)
    username = db.Column(db.String, unique=True, nullable=False)
    email = db.Column(db.String)
```

* Launch Flask Shell with Environment Variables

Set environment variables on Mac.
```shell
export SALES_DB_HOST=localhost
export SALES_DB_PORT=5432
export SALES_DB_NAME=sales_db
export SALES_DB_USER=sales_user
export SALES_DB_PASS=itversity
```

The equivalent commands to set environment variables on Windows.
```powershell
$Env:SALES_DB_HOST = "localhost"
$Env:SALES_DB_PORT = "5432"
$Env:SALES_DB_NAME = "sales_db"
$Env:SALES_DB_USER = "sales_user"
$Env:SALES_DB_PASS = "itversity"
```

After setting required environment variables run `python -m flask shell` to launch `flask shell`.

* Setup Table and Data using Flask Shell

Update `models/user.py` to map user model to users table in Postgres database.
```python
from app import db

class User(db.Model):
    __tablename__ = 'users' # Model and table name are different
    id = db.Column(db.Integer, primary_key=True)
    first_name = db.Column(db.String)
    last_name = db.Column(db.String)
    username = db.Column(db.String, unique=True, nullable=False)
    email = db.Column(db.String)
```

Launch flask shell using `python -m flask shell` and run the below code to reset the table and populate new data.
```python
db.drop_all()
db.create_all()

users = [
    {'first_name': 'Scott', 'last_name': 'Tiger', 'username': 'stiger', 'email': 'stiger@email.com'},
    {'first_name': 'Mickey', 'last_name': 'Mouse', 'username': 'mmouse', 'email': 'mmouse@email.com'},
    {'first_name': 'Charlie', 'last_name': 'Chaplin', 'username': 'cchaplin', 'email': 'cchaplin@email.com'}
]

for user in users:
    user_rec = User(**user)
    db.session.add(user_rec)

db.session.commit()
```

Validate using flask shell to see if the data is inserted or not.
```python
from models.user import User
u = db.session.execute(db.select(User)).fetchone()
u[0].id
u[0].username

user_recs = db.session.query(User).all()
users = list(map(lambda rec: rec.__dict__, user_recs))
print(users)
```


* Run and Validate the Application

Run the application using Flask.
```shell
python -m flask run
```

Go to the new end point http://localhost:5000/users using browser and check if the table is rendered or not.

* Validate using pgAdmin

Login using pgAdmin and check the following:
* In the sidebar navigate to see if the table users is found or not.
* Run queries to see if the data is populated in the database or not (using Query Tool).

```sql
SELECT * FROM users;
```

* Exercise - Flask and Postgres Integration

1. Make sure to create courses table for course model.
2. Populate following data into the table.

|Course Id|Course Name|Course Author|Course Endpoint|
|---|---|---|---|
|1|Mastering Python|Scott Tiger|mastering-python|
|2|Python App Development|Donald Duck|python-app-development|
|3|DevOps Bootcamp|Mickey Mouse|devops-bootcamp|

3. Validate by visiting `courses` end point.
4. Connect to Postgres database using pgAdmin and run query against courses table.

* Solution - Flask and Postgres Integration

Create Course model under `models/course.py`.

```python
from app import db

class Course(db.Model):
    __tablename__ = 'courses'
    course_id = db.Column(db.Integer, primary_key=True)
    course_name = db.Column(db.String)
    course_author = db.Column(db.String)
    course_endpoint = db.Column(db.String, unique=True)
```

Launch flask shell using `python -m flask shell` and create Course Model.
```python
db.create_all()
```

Populate data into the table using model. We will use flask shell to run below python code to populate data into the table.
```python
from models.course import Course

courses = [
    {'course_id': 1, 'course_name': 'Mastering Python', 'course_author': 'Scott Tiger', 'course_endpoint': 'mastering-python'},
    {'course_id': 2, 'course_name': 'Python App Development', 'course_author': 'Donald Duck', 'course_endpoint': 'python-app-development'},
    {'course_id': 3, 'course_name': 'DevOps Bootcamp', 'course_author': 'Mickey Mouse', 'course_endpoint': 'devops-bootcamp'}
]

for item in courses:
    course = Course(**item)
    db.session.add(course)

db.session.commit()
```

Validate by using get API by passing course id.
```python
c = Course.query.get(1)

c.course_id
c.course_name
c.course_endpoint
```